In [136]:
import numpy as np

In [137]:
X = 1500
Y = 1500
Z = 20
K = 3
data = np.random.rand(X,Y,Z)

# Try 1

In [3]:
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

In [116]:
%%prun -l 20
comp_res = {}
for x in range(X): 
    comp_res[x] = {}
    
    for y in range(Y):
        comp_res[x][y] = {}
        col = data[x, y, :]
        
        for x_iter in range(max(0, x-K), min(X, x+K+1)):  #  for x_iter in range(x, min(X, x+K+1)): 
            comp_res[x][y][x_iter] = {}
            
            for y_iter in range(max(0, y-K), min(Y, y+K+1)):

                if x_iter in comp_res: # придумать попроще
                    if y_iter in comp_res[x_iter]:
                        if x in comp_res[x_iter][y_iter]:
                            if y in comp_res[x_iter][y_iter][x]:
                                comp_res[x][y][x_iter][y_iter] = comp_res[x_iter][y_iter][x][y]
                                continue

                if (x_iter != x) or (y_iter != y): # подумать над форматом кэширования
                    comp_res[x][y][x_iter][y_iter] = vectors_comparison(col, data[x_iter, y_iter])

neighbours_comp = np.zeros((X,Y)).reshape(X,Y)
for x in range(X):
    for y in range(Y):
        tmp_arr = []
        for x_iter in comp_res[x][y]:
            for y_iter in comp_res[x][y][x_iter]:
                tmp_arr.append(comp_res[x][y][x_iter][y_iter])
        neighbours_comp[x][y] = np.mean(tmp_arr)

         260 function calls in 0.002 seconds

   Ordered by: internal time
   List reduced from 22 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.002    0.002 {built-in method builtins.exec}
        1    0.001    0.001    0.001    0.001 <string>:1(<module>)
        9    0.000    0.000    0.000    0.000 _methods.py:143(_mean)
        9    0.000    0.000    0.000    0.000 {method 'reduce' of 'numpy.ufunc' objects}
        9    0.000    0.000    0.000    0.000 {built-in method numpy.array}
        9    0.000    0.000    0.000    0.000 fromnumeric.py:3269(mean)
        9    0.000    0.000    0.000    0.000 _methods.py:59(_count_reduce_items)
       20    0.000    0.000    0.000    0.000 <timed exec>:1(vectors_comparison)
        9    0.000    0.000    0.001    0.000 <__array_function__ internals>:2(mean)
        9    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.impleme

# Try 2

In [6]:
%%time

res = np.zeros(data.shape[0:2])
for x in range(X):
    for y in range(Y):        
        col = data[x, y, :]        
        for x_idx in range(-K, K+1):
            for y_idx in range(-K, K+1):
                if (0 <= x + x_idx < X) and (0 <= y + y_idx < Y):
                    res[x, y] += vectors_comparison(col, data[x + x_idx, y + y_idx])
        res[x, y] /= (min(x + x_idx, X - 1) - max(0, x - x_idx) + 1) * (min(y + y_idx, Y - 1) - max(0, y - y_idx) + 1) - 1

CPU times: user 5min 44s, sys: 34.9 ms, total: 5min 44s
Wall time: 5min 44s


# Try 3

In [138]:
import numba

In [139]:
from numba import njit

In [143]:
X = np.int32(1500)
Y = np.int32(1500)
Z = np.int32(20)
K = np.int32(3)
data = np.random.rand(X,Y,Z).astype('float32')

In [144]:
%%time

@njit(numba.types.float32(numba.types.float32[:], 
                          numba.types.float32[:]),
      fastmath=True)
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit(numba.types.float32[:, :](numba.types.float32[:, :, :], 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32),
    locals={'res': numba.types.float32[:, :],
            'col': numba.types.float32[:]}, parallel=True)
def get_comparison(data, X, Y, Z, K):
    res = np.zeros(data.shape[0:2], dtype='float32')
    for x in range(X):
        for y in range(Y):        
            col = data[x, y, :]        
            for x_idx in range(-K, K+1):
                for y_idx in range(-K, K+1):
                    sss = np.float32(0.0)
                    if (0 <= x + x_idx < X) and (0 <= y + y_idx < Y):
                        sss += vectors_comparison(col, data[x + x_idx, y + y_idx])
            count = (min(x + x_idx, X - 1) - max(0, x - x_idx) + 1) * (min(y + y_idx, Y - 1) - max(0, y - y_idx) + 1) - 1
            res[x, y] = sss / count
    return res

CPU times: user 932 ms, sys: 12.1 ms, total: 944 ms
Wall time: 943 ms


In [145]:
%%time
res = get_comparison(data, X, Y, Z, K)

CPU times: user 16.6 s, sys: 19.2 ms, total: 16.6 s
Wall time: 16.5 s


In [ ]:
%%time

@njit(numba.types.float32(numba.types.float32[:], 
                          numba.types.float32[:]),
      fastmath=True)
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit(numba.types.float32[:, :](numba.types.float32[:, :, :], 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32),
    locals={'res': numba.types.float32[:, :],
            'col': numba.types.float32[:]}, parallel=True)
def get_comparison(data, X, Y, Z, K):
    res = np.zeros(data.shape[0:2], dtype='float32')
    data = np.pad(data, K, constant_values=0)[1:-1] # подумать
    data_shifted = 
    for x in range(X):
        for y in range(Y):        
            col = data[x, y, :]        
            for x_idx in range(-K, K+1):
                for y_idx in range(-K, K+1):
                    sss = np.float32(0.0)
                    if (0 <= x + x_idx < X) and (0 <= y + y_idx < Y):
                        sss += vectors_comparison(col, data[x + x_idx, y + y_idx])
            count = (min(x + x_idx, X - 1) - max(0, x - x_idx) + 1) * (min(y + y_idx, Y - 1) - max(0, y - y_idx) + 1) - 1
            res[x, y] = sss / count
    return res

In [374]:
X, Y, Z, K = 5, 5, 3, 2

data = np.random.randint(2, 10, (X, Y, Z))

In [375]:
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

def get_comparison(data, X, Y, Z, K):
    res = np.zeros(data.shape[0:2], dtype='float32')
    for x in range(X):
        for y in range(Y):  
            col = data[x, y, :]        
            sss = np.float32(0.0)
            for x_idx in range(-K, K+1):
                for y_idx in range(-K, K+1): 
                    if (0 <= x + x_idx < X) and (0 <= y + y_idx < Y) and ((x_idx!=0) or (y_idx!=0)):
                        sss += vectors_comparison(col, data[x + x_idx, y + y_idx])
            #count = (min(x + x_idx, X - 1) - max(0, x - x_idx) + 1) * (min(y + y_idx, Y - 1) - max(0, y - y_idx) + 1) - 1
            res[x, y] = sss # / count
    return res

In [376]:
result1 = get_comparison(data.copy(), X, Y, Z, K)

In [377]:
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2), axis = 2)
    return res

counter = 0
a_pad = np.pad(data, K)
res = np.zeros(a_pad.shape[:2])
for xk in range(0, K+1):
    for yk in range(0, K+1):
        if (xk!=0) or (yk!=0):      
            counter += 2
            shifted_arr = np.roll(np.roll(a_pad, -yk, axis = 1), -xk, axis = 0)
            r1 = vectors_comparison(a_pad, shifted_arr) 
            r1_inv = np.roll(np.roll(r1, yk, axis = 1), xk, axis = 0)
            res += r1 + r1_inv
            if (yk!=0) and (xk!=0):
                counter += 2
                shifted_arr = np.roll(np.roll(a_pad, yk, axis = 1), -xk, axis = 0)
                r1 = vectors_comparison(a_pad, shifted_arr) 
                r1_inv = np.roll(np.roll(r1, -yk, axis = 1), xk, axis = 0)
                res += r1 + r1_inv
print(counter)

24


In [378]:
result2 = res[K:-K, K:-K]

In [379]:
np.all(result1==result2)

True